In [1]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

   Updating git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`


   Updating registry at `C:\Users\sousa\.julia\registries\General`



Authentication required.
Opening https://pkg.juliahub.com/auth/response?28b42c610386ae1dadced4006998a2d5 to authenticate.

Authentication timed out. Please try again.

Alternatively, open

https://pkg.juliahub.com/auth

in a browser, authenticate, and save the downloaded file at

C:\Users\sousa\.julia\servers\pkg.juliahub.com\auth.toml

Press Enter when you're done...


Authentication required.
Opening https://pkg.juliahub.com/auth/response?aeee0d8ae140ced5f04387e31d0a5f81 to authenticate.



┌ Error: JuliaTeam authentication handler failed.
└ @ Main C:\Users\sousa\AppData\Local\JuliaPro-1.5.3-1\Julia-1.5.3\etc\julia\startup.jl:26
┌ Warning: could not download https://pkg.juliahub.com/registries
└ @ Pkg.Types C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\Pkg\src\Types.jl:951
   Updating registry at `C:\Users\sousa\.julia\registries\JuliaComputingRegistry`
  Resolving package versions...
No Changes 

LoadError: InterruptException:

# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can
lead to long discussions, first as to exactly what is being benchmarked and
secondly what explains the differences.  These simple questions can sometimes
get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for
yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT:
[Boxes and registers](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)).

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - Julia (built-in)
    - Julia (hand-written)
    - C (hand-written)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
- Towards exploiting parallelism with Julia
    - Allowing for floating point associativity
    - Making use of four cores at once: built-in
    - Making use of four cores at once: hand-written
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.060110004256964666
 0.5075683180456987
 0.7636466694987871
 0.6607076459441128
 0.13507532380562526
 0.41564505104428173
 0.614973115832165
 0.07046585215131396
 0.7505256378959506
 0.7800016074961564
 0.7544460660951431
 0.8903312171950961
 0.8658310960102851
 ⋮
 0.7046125024046077
 0.05148866164628507
 0.8510370827407661
 0.4234616805670315
 0.31963901571974396
 0.36369666616688456
 0.8455728727601841
 0.27424443210282523
 0.31273165168030803
 0.6725592693063593
 0.6035733570002384
 0.27819316199496136

In [2]:
sum(a)

5.000992326781889e6

The expected result is ~0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [4]:
@time sum(a)

  0.012496 seconds (1 allocation: 16 bytes)


5.000351514035497e6

In [5]:
@time sum(a)

  0.006031 seconds (1 allocation: 16 bytes)


5.000351514035497e6

In [6]:
@time sum(a)

  0.006442 seconds (1 allocation: 16 bytes)


5.000351514035497e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [3]:
using BenchmarkTools

In [4]:
@benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.319 ms (0.00% GC)
  median time:      6.069 ms (0.00% GC)
  mean time:        6.690 ms (0.00% GC)
  maximum time:     38.263 ms (0.00% GC)
  --------------
  samples:          747
  evals/sample:     1

# 1. Julia Built-in

So that's the performance of Julia's built-in sum — but that could be doing any number of tricks to be fast, including not using Julia at all in the first place! Of course, it is indeed written in Julia, but would it perform if we write a naive implementation ourselves?

In [5]:
@which sum(a)

sum(a::AbstractArray; dims) in Base at reducedim.jl:722

Let's save these benchmark results to a dictionary so we can start keeping track of them and comparing them down the line.

In [6]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.651 ms (0.00% GC)
  median time:      8.023 ms (0.00% GC)
  mean time:        9.029 ms (0.00% GC)
  maximum time:     29.978 ms (0.00% GC)
  --------------
  samples:          554
  evals/sample:     1

In [13]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 1 entry:
  "Julia built-in" => 5.651

# 2. Julia (hand-written)

In [14]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [15]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.026 ms (0.00% GC)
  median time:      12.712 ms (0.00% GC)
  mean time:        12.988 ms (0.00% GC)
  maximum time:     21.073 ms (0.00% GC)
  --------------
  samples:          385
  evals/sample:     1

In [16]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "Julia hand-written" => 12.0256
  "Julia built-in"     => 5.651

So that's about 2x slower than the builtin definition. We'll see why later on.

But first: is this fast?  How would we know?  Let's compare it to some other languages...

#  3. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [17]:
using Libdl

In [18]:

C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [19]:
c_sum(a)

LoadError: could not load library "C:\Users\sousa\AppData\Local\Temp\jl_WsKMrQXsBj"
%1 não é um aplicativo Win32 válido. 

In [ ]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

We can now benchmark the C code directly from Julia:

In [ ]:
c_bench = @benchmark c_sum($a)

In [21]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

# 4. Python's built in `sum`

The `PyCall` package provides a Julia interface to Python:

In [20]:
using PyCall

In [21]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [22]:
pysum(a)

5.000992326781932e6

In [23]:
pysum(a) ≈ sum(a)

true

In [24]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     1.481 s (0.00% GC)
  median time:      1.500 s (0.00% GC)
  mean time:        1.499 s (0.00% GC)
  maximum time:     1.514 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [25]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "Julia hand-written" => 12.0256
  "Julia built-in"     => 5.651
  "Python built-in"    => 1481.04

# 5. Python: `numpy`

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [26]:
using Conda

In [27]:
# Conda.add("numpy")

In [28]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     11.046 ms (0.00% GC)
  median time:      11.650 ms (0.00% GC)
  mean time:        11.786 ms (0.00% GC)
  maximum time:     25.217 ms (0.00% GC)
  --------------
  samples:          425
  evals/sample:     1

In [29]:
numpy_sum(a)

5.0009923267818885e6

In [30]:
numpy_sum(a) ≈ sum(a)

true

In [31]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "Python numpy"       => 11.0456
  "Julia hand-written" => 12.0256
  "Julia built-in"     => 5.651
  "Python built-in"    => 1481.04

# 6. Python, hand-written

In [32]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x0000000002658DC0>

In [33]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     2.093 s (0.00% GC)
  median time:      2.151 s (0.00% GC)
  mean time:        2.138 s (0.00% GC)
  maximum time:     2.169 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1

In [34]:
sum_py(a)

5.000992326781932e6

In [35]:
sum_py(a) ≈ sum(a)

true

In [36]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "Python numpy"        => 11.0456
  "Julia hand-written"  => 12.0256
  "Python hand-written" => 2093.13
  "Julia built-in"      => 5.651
  "Python built-in"     => 1481.04

# Summary so far

In [37]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia built-in..............5.7
Python numpy...............11.0
Julia hand-written.........12.0
Python built-in..........1481.0
Python hand-written......2093.1


We seem to have three different performance classes here: The numpy and Julia
builtins are leading the pack, followed by the hand-written Julia and C
definitions. Those seem to be about 2x slower.  And then we have the much much
slower Python definitions over 100x slower.

# Exploiting parallelism with Julia

The fact that our hand-written Julia solution was almost an even multiple of
2x slower than the builtin solutions is a big clue: perhaps theres some sort
of 2x parallelism going on here?

(In fairness, there are ways to exploit parallelism in other languages, too,
but for brevity we won't cover them)

# 7. Julia (allowing floating point associativity)

The `for` loop

```julia
for a in A
    s += a
end
```

defines a very strict _order_ to the summation: Julia follows exactly what you
wrote and adds the elements of `A` to the result `s` in the order it iterates.
Since floating point numbers aren't associative, a rearrangement here would
change the answer — and Julia is loathe to give you different answer than
the one you asked for.

You can, however, tell Julia to relax that rule and allow for associativity
with the `@fastmath` macro. This might allow Julia to rearrange the sum in an
advantageous manner.

In [38]:
function mysum_fast(A)
    s = 0.0
    for a in A
        @fastmath s += a
    end
    s
end

mysum_fast (generic function with 1 method)

In [39]:
j_bench_hand_fast = @benchmark mysum_fast($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.159 ms (0.00% GC)
  median time:      5.504 ms (0.00% GC)
  mean time:        5.579 ms (0.00% GC)
  maximum time:     12.262 ms (0.00% GC)
  --------------
  samples:          896
  evals/sample:     1

In [40]:
mysum_fast(a)

5.000992326781954e6

In [41]:
d["Julia hand-written fast"] = minimum(j_bench_hand_fast.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "Python numpy"            => 11.0456
  "Julia hand-written"      => 12.0256
  "Python hand-written"     => 2093.13
  "Julia built-in"          => 5.651
  "Python built-in"         => 1481.04
  "Julia hand-written fast" => 5.1591

# 8. Distributed Julia (built-in)

We can take this one step further: nearly every modern computer these days has
multiple cores. All the above solutions are working one core hard, but all the
others are just sitting by idly. Let's put them to work!

In [43]:
import Pkg; Pkg.add("DistributedArrays")

   Updating registry at `C:\Users\sousa\.julia\registries\General`
   Updating registry at `C:\Users\sousa\.julia\registries\JuliaComputingRegistry`
  Resolving package versions...
Updating `C:\Users\sousa\.julia\environments\v1.5\Project.toml`
  [aaf54ef3] + DistributedArrays v0.6.5
Updating `C:\Users\sousa\.julia\environments\v1.5\Manifest.toml`
  [aaf54ef3] + DistributedArrays v0.6.5
  [27ebfcd6] + Primes v0.4.0


In [ ]:
using Distributed
using DistributedArrays
addprocs(4)
@sync @everywhere workers() include("/opt/julia-1.0/etc/julia/startup.jl") # Needed just for JuliaBox
@everywhere using DistributedArrays

┌ Info: Precompiling DistributedArrays [aaf54ef3-cdf8-58ed-94cc-d582ad619b94]
└ @ Base loading.jl:1278


In [ ]:
adist = distribute(a)
j_bench_dist = @benchmark sum($adist)

In [ ]:
d["Julia 4x built-in"] = minimum(j_bench_dist.times) / 1e6
d

# 9. Distributed Julia (hand-written)

Ok, that might be cheating, too — it's again just calling a library
function. Is it possible to write distributed sum ourselves?

In [ ]:
function mysum_dist(a::DArray)
    r = Array{Future}(undef, length(procs(a)))
    for (i, id) in enumerate(procs(a))
        r[i] = @spawnat id sum(localpart(a))
    end
    return sum(fetch.(r))
end

In [ ]:
j_bench_hand_dist = @benchmark mysum_dist($adist)

In [ ]:
d["Julia 4x hand-written"] = minimum(j_bench_hand_dist.times) / 1e6
d

# Overall Summary

In [ ]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

# Key take-aways

* Julia allows for serial C-like performance, even with hand-written functions
* Julia allows us to exploit many forms of parallelism to further improve performance. We demonstrated:
    * Single-processor parallelism with SIMD
    * Multi-process parallelism with DistributedArrays
* But there are many other ways to express parallelism, too!